# Search

This notebook describes how you can search a program space as defined by a grammar.
Specifically, we will look at example-based search, where the goal is to find a program that is able to transform the inputs of every example to the corresponding output.

### Setup

In [1]:
include("../src/HerbExamples.jl")
using .Herb

### Defining the program space

First, we need to define the program space using a grammar. If you are not familiar with how this works, please have a look at the notebook for creating grammars.

Here, we create a basic grammar with a few arithmetic operators and a single variable `x`.

In [2]:
g₁ = Herb.Grammars.@cfgrammar begin
    Real = |(0:9)
    Real = x
    Real = Real + Real
    Real = Real - Real
    Real = Real * Real
end

1: Real = 0
2: Real = 1
3: Real = 2
4: Real = 3
5: Real = 4
6: Real = 5
7: Real = 6
8: Real = 7
9: Real = 8
10: Real = 9
11: Real = x
12: Real = Real + Real
13: Real = Real - Real
14: Real = Real * Real


### Defining the problem

As mentioned before, we are looking at example-based search. 
This means that the problem is defined by a set of input-output examples. 
An input-output example consists of an input and an output, as the name suggests.
The input is defined as a dictionary, with a value assigned to each variable in the grammar.
It is important to write the variable name as a `Symbol` instead of a string.
A `Symbol` in Julia is written with a colon prefix, i.e. `:x`. 
The output of the input-output example is just a single value.

In the cell below we automatically generate some examples for x-values 1-5.

In [3]:
# Create input-output examples
examples = [Herb.Data.IOExample(Dict(:x => x), 3x+5) for x ∈ 1:5]

5-element Vector{Main.Herb.Data.IOExample}:
 Main.Herb.Data.IOExample(Dict(:x => 1), 8)
 Main.Herb.Data.IOExample(Dict(:x => 2), 11)
 Main.Herb.Data.IOExample(Dict(:x => 3), 14)
 Main.Herb.Data.IOExample(Dict(:x => 4), 17)
 Main.Herb.Data.IOExample(Dict(:x => 5), 20)

Now that we have some input-output examples, we can define the problem. 
Next to the examples, a problem also contains a name, which can be used to keep track of them. 
For now, this is irrelevant, and you can give the program any name you like.

In [4]:
problem = Herb.Data.Problem(examples, "example")

Main.Herb.Data.Problem(Main.Herb.Data.Example[Main.Herb.Data.IOExample(Dict(:x => 1), 8), Main.Herb.Data.IOExample(Dict(:x => 2), 11), Main.Herb.Data.IOExample(Dict(:x => 3), 14), Main.Herb.Data.IOExample(Dict(:x => 4), 17), Main.Herb.Data.IOExample(Dict(:x => 5), 20)], "example")

### Searching

Now that we have defined the search space and the goal of the search, we can start the search. 

Of course, our problem is underdefined; there might be multiple programs that satisfy our examples. 
Consider the case where we also have a ternary operator and some boolean operators in our grammar: we could synthesize the program `x ≤ 5 ? 3x+5 : 0`. 
This program satisfies all our examples, but we don't expect it to generalize very well.

In general, we assume that a smaller program is more general than a larger program. 
Therefore, we search for the smallest program in our grammar that satisfies our examples. 
This can be done using a breadth-first search over the program/search space.

This search is very basic; it makes use of an enumeration technique, where we enumerate programs one-by-one until we find a program that matches our examples.

Such a search is done by passing the grammar, the problem, the maximum depth of the programs we want to search for (3), and a constructor for a breadth-first search enumerator to the basic enumerative search procedure:

In [5]:
Herb.Search.search(g₁, problem, 3, Herb.Search.get_bfs_enumerator)

:(3x + 5)

As you can see, the search procedure found the correct program!